# Module 3 R Notebook
# Finding Differentially Expressed Genes

## Important note - read first

In this course, you are not required to become proficient in writing R scripts for Big Biomedical Data Analytics. The assignments will be based on each module's template, and replacing some parts of them with new data and parameters so that you will get new results and interpret them. 

Understanding all the coding details in R will depend on your bakcground knowledge in programming concepts. 

Therefore if you do not understand all the coding details in R, try to run the code and understand mostly: 

1) what information do I have to provide to this script to make it work ?

2) how to execute this script (it will always be: run the cell containing the script Cell --> Run cells, or click on SHIFT + ENTER at the same time, or simply click on the right arrow icon) ?

3) what results do I get ?

4) how to interpret the results ?

Little by little, you will become familiar with R programming language and, example after example, you will understand the scripts and be ready for more advanced programming work. 

## Assignment goals

In this notebook, you are going to practice selecting features with R scripts, and particularly differentially expressed genes. This implies selecting features representing genes expressed differently between two groups, such as a normal group and a disease group. 

One strategy would be to compare gene expressions between a group of patients and a standard normal individual or group of normal individuals. 

Here we are choosing a breast cancer dataset downloaded from Firehose (https://gdac.broadinstitute.org/). These are next generation sequncing data that are provided already normalized. 

So let us get started !

## Preparing the environment

Good news, we do not have to use external libraries for this analysis !


## Loading the data

The dataset we are loading is called  "BRCA.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt". 

It looks like this: 

Hybridization REF TCGA.3C.AAAU.01A.11R.A41B.07 TCGA.3C.AALI.01A.11R.A41B.07 ...

gene_id                normalized_count             normalized_count        ...                      

?|100130426                 0.0000                       0.0000             ...        

This dataset has genes as rows and patients as columns. Patient IDs refer to TCGA IDs such as TCGA.3C.AAAU.01A.11R.A41B.07, etc. The top two lines of the file are header information, and we direct 'read.table' to skip these two lines since we only want the gene expressions to be placed in a dataframe we call 'mrnaNorm'. 
The first column of 'mrnaNorm'  contains the list of genes measured.

Since this is a large datset, you will have to wait until the '[*]' becomes a number in cell #1 to progress to the next cell.

In [18]:
# cell #1
mrnaNorm <- read.table("/data/BRCA.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt", 
            header = F, fill = T, skip = 2)
class(mrnaNorm)

[1] "data.frame"

We also want the patient IDs in the first row, which is why we read only this first line and place it in a second dataframe callses 'mrnaIDs'. 
We also remove the first column with '[, -1]', the one entitled 'Hybridization REF', because it is not a patientID.

In [3]:
# cell #2
mrnaIDs <- read.table("/data/BRCA.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt", 
            header = F, fill = T, nrows = 1)
class(mrnaIDs)
dim(mrnaIDs)
mrnaIDs <- mrnaIDs[, -1][, -1]  # remove the first solumn "Hybridization REF" which spans two columns because of the space between the two words
dim(mrnaIDs)

[1] "data.frame"

[1]    1 1214

[1]    1 1212

## Data preprocessing

Next we take a look at our data. Since this is a very large dataset, we are not going to display all the rows and columns, but only the first 5 rows and columns.
'mrnaNorm' contains 20,531 rows, representing the genes and their expressions, and 1213 columns (1212 subjects and the first column containing gene names).

In [4]:
# cell #3
dim(mrnaNorm)
dim(mrnaIDs)
mrnaIDs5 = mrnaIDs[,c(1,2,3,4,5)] # extract first 5 patientIDs
mrnaNorm5x5 = mrnaNorm[1:5, 1:5] # first 5 rows and columns 
head(mrnaIDs5, 2) # display first two rows
head(mrnaNorm, 2) # display first two rows
summary(mrnaNorm5x5) # summary statistics

ERROR: Error in eval(expr, envir, enclos): object 'mrnaNorm' not found


[1]    1 1212

ERROR: Error in eval(expr, envir, enclos): object 'mrnaNorm' not found


,V3,V4,V5,V6,V7
1,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07


ERROR: Error in head(mrnaNorm, 2): object 'mrnaNorm' not found


ERROR: Error in summary(mrnaNorm5x5): object 'mrnaNorm5x5' not found


The following instruction analyzes the patient IDs because the IDs can tell us whether a particular column is from a normal sample or a tumor sample. 
TCGA has data both from tumor samples and from normal samples taken from the same patient. This information can be found in the label of the column. For example in ID:
TCGA.3C.AAAU.01A.11R.A41B.07 
the type of sample is indicated in the 4th group: 01A.
Tumor types range from 01 - 09, normal types from 10 - 19 and control samples from 20 - 29.
Therefore we are extracting in the cell below this code indicating whether the sample is from normal or not. 
This group is the fourth in the ID character string, obtained with 'strsplit', and we get the two characters of interest by taking the first two characters with 'substr'.
We do this for all the IDs by applying with 'lapply' the same process to all the IDs in 'mrnaIDs'.

In [5]:
# cell #4
samp <- lapply(as.list(t(mrnaIDs)), function(t) substr(unlist(strsplit(t, "-"))[4], 1, 2))
# extract the sample type (tumor / normal), excluding the first column since it is a gene ID
sampleType <- as.data.frame(samp)
dim(sampleType)
# 1 1212

[1]    1 1212

Next we are going to count how many normals and how many tumor samples we have. 
The function 'unique' gives us the different types we have. 
The function 'table' calculates how many samples we have of each type. We see that we have 1100 of type '01' or '06' for tumor, and 112 of type '11' for normal.

In [6]:
# cell #5
unique(sampleType)
# extracts how many unique objects there are 

tab <- table(unlist(sampleType))
tab
# count how many of each type
#   01   06   11 
#  1093   7  112 

,X.01.,X.01..1,X.01..2,X.01..3,X.01..4,X.01..5,X.01..6,X.01..7,X.01..8,X.01..9,X.01..10,X.01..11,X.01..12,X.01..13,X.01..14,X.01..15,X.01..16,X.01..17,X.01..18,X.01..19,X.01..20,X.01..21,X.01..22,X.01..23,X.01..24,X.01..25,X.01..26,X.01..27,X.01..28,X.01..29,X.01..30,X.01..31,X.01..32,X.01..33,X.01..34,X.01..35,X.01..36,X.01..37,X.01..38,X.01..39,X.01..40,X.01..41,X.01..42,X.01..43,X.01..44,X.01..45,X.01..46,X.01..47,X.01..48,X.01..49,X.01..50,X.01..51,X.01..52,X.01..53,X.01..54,X.01..55,X.01..56,X.01..57,X.01..58,X.01..59,X.01..60,X.01..61,X.01..62,X.01..63,X.01..64,X.01..65,X.01..66,X.01..67,X.01..68,X.01..69,X.01..70,X.01..71,X.01..72,X.01..73,X.01..74,X.01..75,X.01..76,X.01..77,X.01..78,X.01..79,X.01..80,X.01..81,X.01..82,X.01..83,X.01..84,X.01..85,X.01..86,X.01..87,X.01..88,X.01..89,X.01..90,X.01..91,X.01..92,X.01..93,X.01..94,X.01..95,X.01..96,X.01..97,X.01..98,X.01..99,X.01..100,X.01..101,X.01..102,X.01..103,X.01..104,X.01..105,X.01..106,X.01..107,X.01..108,X.01..109,X.01..110,X.01..111,X.01..112,X.01..113,X.01..114,X.01..115,X.01..116,X.01..117,X.01..118,X.01..119,X.01..120,X.01..121,X.01..122,X.01..123,X.01..124,X.11.,X.01..125,X.01..126,X.11..1,X.01..127,X.01..128,X.11..2,X.01..129,X.01..130,X.11..3,X.01..131,X.01..132,X.11..4,X.01..133,X.11..5,X.01..134,X.11..6,X.01..135,X.01..136,X.01..137,X.01..138,X.01..139,X.01..140,X.01..141,X.01..142,X.01..143,X.01..144,X.01..145,X.01..146,X.01..147,X.01..148,X.01..149,X.01..150,X.01..151,X.01..152,X.01..153,X.01..154,X.01..155,X.01..156,X.01..157,X.01..158,X.01..159,X.01..160,X.01..161,X.01..162,X.01..163,X.01..164,X.01..165,X.01..166,X.01..167,X.01..168,X.01..169,X.01..170,X.01..171,X.01..172,X.01..173,X.01..174,X.01..175,X.01..176,X.01..177,X.01..178,X.01..179,X.01..180,X.01..181,X.01..182,X.01..183,X.01..184,X.01..185,X.01..186,X.01..187,X.01..188,X.01..189,X.01..190,X.01..191,X.01..192,X.01..193,X.01..194,X.01..195,X.01..196,X.01..197,X.01..198,X.01..199,X.01..200,X.01..201,X.01..202,X.01..203,X.01..204,X.01..205,X.01..206,X.01..207,X.01..208,X.01..209,X.01..210,X.01..211,X.01..212,X.01..213,X.01..214,X.01..215,X.01..216,X.01..217,X.01..218,X.01..219,X.01..220,X.01..221,X.01..222,X.01..223,X.01..224,X.01..225,X.01..226,X.01..227,X.01..228,X.01..229,X.01..230,X.01..231,X.01..232,X.01..233,X.01..234,X.01..235,X.01..236,X.01..237,X.01..238,X.01..239,X.01..240,X.01..241,X.01..242,X.01..243,X.01..244,X.01..245,X.01..246,X.01..247,X.01..248,X.01..249,X.01..250,X.01..251,X.11..7,X.01..252,X.01..253,X.01..254,X.01..255,X.11..8,X.01..256,X.01..257,X.11..9,X.01..258,X.01..259,X.01..260,X.11..10,X.01..261,X.01..262,X.01..263,X.01..264,X.01..265,X.01..266,X.01..267,X.01..268,X.01..269,X.01..270,X.01..271,X.01..272,X.01..273,X.01..274,X.01..275,X.01..276,X.01..277,X.01..278,X.01..279,X.01..280,X.01..281,X.01..282,X.01..283,X.01..284,X.01..285,X.01..286,X.01..287,X.06.,X.01..288,X.01..289,X.01..290,X.01..291,X.01..292,X.01..293,X.01..294,X.01..295,X.01..296,X.01..297,X.01..298,X.01..299,X.01..300,X.01..301,X.01..302,X.01..303,X.01..304,X.01..305,X.01..306,X.01..307,X.01..308,X.01..309,X.01..310,X.01..311,X.01..312,X.01..313,X.01..314,X.01..315,X.01..316,X.01..317,X.01..318,X.01..319,X.01..320,X.01..321,X.01..322,X.01..323,X.01..324,X.01..325,X.01..326,X.01..327,X.01..328,X.01..329,X.01..330,X.01..331,X.01..332,X.01..333,X.01..334,X.01..335,X.01..336,X.01..337,X.01..338,X.01..339,X.01..340,X.01..341,X.01..342,X.01..343,X.01..344,X.01..345,X.01..346,X.01..347,X.01..348,X.01..349,X.01..350,X.01..351,X.01..352,X.01..353,X.01..354,X.01..355,X.01..356,X.01..357,X.01..358,X.01..359,X.01..360,X.01..361,X.01..362,X.01..363,X.01..364,X.01..365,X.01..366,X.01..367,X.01..368,X.01..369,X.01..370,X.01..371,X.01..372,X.01..373,X.01..374,X.01..375,X.01..376,X.01..377,X.01..378,X.01..379,X.01..380,X.01..381,X.01..382,X.01..383,X.01..384,X.01..385,X.01..386,X.01..387,X.01..388,X.01..389,X.01..390,X.01..391,X.01..392,X.01..393,X.01..394,X.01..395,X.01..396,X.01..397,X.01..398,X.01..399,X.01..400,X.01..40


  01   11   06 
1093  112    7 

We are going to associate a class of '1' for the tumor samples and of '0' for the normal samples. We check that we have 112 normals (class '0') and 1100 tumors (class '1').
We do this for all the types by applying with 'lapply' the same process to all the types in 'samp'.

In [13]:
# cell #6
sampClass <- lapply(samp, function(t) (if (t < 10) return("1") else return("0")))
mrnaClass <- as.data.frame(sampClass)
dim(mrnaClass)
table(unlist(sampClass))
#   0    1 
# 112 1100 

[1]    1 1212


   0    1 
 112 1100 

Since in some cases we may need the class as a number (and not a character), we create as well a list of classes as numbers.

In [14]:
# cell #7
sampClassNum <- lapply(samp, function(t) (if (t < 10) return(1) else return(0)))
mrnaClassNum <- as.data.frame(sampClassNum) 

Here I provide a function that calculates a ranking of features (genes in this case) with the BSS/WSS method. You do not need to understand all the details - only to execute the following cell. This will make the bssWssFast function available for later processing.

In [25]:
# cell #8
bssWssFast <- function (X, givenClassArr, numClass=2)
# between squares / within square feature selection
{
	classVec <- matrix(0, numClass, length(givenClassArr))
	for (k in 1:numClass) {
		temp <- rep(0, length(givenClassArr))
		temp[givenClassArr == (k - 1)] <- 1
		classVec[k, ] <- temp
	}
	classMeanArr <- rep(0, numClass)
	ratio <- rep(0, ncol(X))
	for (j in 1:ncol(X)) {
		overallMean <- sum(X[, j]) / length(X[, j])
		for (k in 1:numClass) {
			classMeanArr[k] <- 
				sum(classVec[k, ] * X[, j]) / sum(classVec[k, ])
		}
	  classMeanVec <- classMeanArr[givenClassArr + 1]
	  bss <- sum((classMeanVec - overallMean)^2)
	  wss <- sum((X[, j] - classMeanVec)^2)
	  ratio[j] <- bss/wss
	}
      sort(ratio, decreasing = TRUE, index = TRUE)
}

## Feature selection with BSS/WSS

We are now ready to select features. 

We invoke the bssWSSFast function by passing to it the dataframe of gene expressions 'mrnaNorm' (all numeric, after excluding the first column, which is not numeric since it contains gene names) and another dataframe of classes associated with each sample. Note that we also transpose the 'mrnaNprm' dataframe because we watb to select genes, therefore the genes need to be feature / columns, while in 'mrnaNorm' genes are rows. By transposing 'mrnaNorm' and excluding the first column, we obtain a dataframe that looks like this ('t(mrnaNorm[, -1])'):

             gene1           gene2            gene3        ...
    
patient1   expression 1     expression 2      expression3    ...

patient2   expression 1     expression 2      expression3    ...


bssWssFast will return a dataframe with the features/genes column numbers ranked by decreasing order of importance according to bssWss ratio. 

bss$ix[1] refers to column 'ix' of bss, and could also be expressed by bss[,ix][1]. column 'ix' contains a list of indexes in 'mrnaNorm'.


In [ ]:
# cell #9
bss <- bssWssFast(t(mrnaNorm[, -1]), t(mrnaClassNum), 2)
# returns a list with two elements: a list of bss/wss ratios in decreasing order bss[1] and a list of features bss[2]

bss$ix[1:51]
# show list of 50 ranked gene indexes

We can then list the genes by their name as provided in the 'mrnaNorm' dataset, where the first column is the list of gene names. The names have two codes separated by character '|':

HUGO code
Entrez code

For example, FHL1|2273 is gene with HUGO code FHL1 and Entrez code 2273.

We will continue working on these genes in a future module.

In [24]:
# cell #10
genes <- mrnaNorm[bss$ix[1:51],1]
genes
# show list of 50 tanked gene names
# [1] FHL1|2273        ADAMTS4|9507     ABAT|18          PAG1|55824       ARHGAP1|392     
# [6] TP53TG3B|729355  SCRN3|79634      TIE1|7075        SNRPA|6626       SCCPDH|51097    
#[11] PCYT1B|9468      DSE|29940        LGI4|163175      LIX1L|128077     LCE3D|84648     
#[16] POTEE|445582     GIMAP2|26157     CABYR|26256      PLGLA|285189     LRRC57|255252   
#[21] C21orf33|8209    CDC14A|8556      CD300LF|146722   CEP57|9702       MAGEC2|51438    
#[26] LGALS3|3958      CAT|847          LOC647979|647979 ABCA8|10351      PPIL3|53938     
#[31] EFCAB1|79645     CCDC45|90799     ABCA5|23461      CAV1|857         TP53I3|9540     
#[36] ADRB1|153        SOD1|6647        LDLRAP1|26119    LYPLA1|10434     KLHL10|317719   
#[41] HLA-DPB1|3115    SGCG|6445        MLL2|8085        FGD4|121512      CD33|945        
#[46] BMS1|9790        MARCKSL1|65108   BTNL8|79908      SS18L2|51188     ABCA7|10347   


[1] FHL1|2273        ADAMTS4|9507     ABAT|18          PAG1|55824      
 [5] ARHGAP1|392      TP53TG3B|729355  SCRN3|79634      TIE1|7075       
 [9] SNRPA|6626       SCCPDH|51097     PCYT1B|9468      DSE|29940       
[13] LGI4|163175      LIX1L|128077     LCE3D|84648      POTEE|445582    
[17] GIMAP2|26157     CABYR|26256      PLGLA|285189     LRRC57|255252   
[21] C21orf33|8209    CDC14A|8556      CD300LF|146722   CEP57|9702      
[25] MAGEC2|51438     LGALS3|3958      CAT|847          LOC647979|647979
[29] ABCA8|10351      PPIL3|53938      EFCAB1|79645     CCDC45|90799    
[33] ABCA5|23461      CAV1|857         TP53I3|9540      ADRB1|153       
[37] SOD1|6647        LDLRAP1|26119    LYPLA1|10434     KLHL10|317719   
[41] HLA-DPB1|3115    SGCG|6445        MLL2|8085        FGD4|121512     
[45] CD33|945         BMS1|9790        MARCKSL1|65108   BTNL8|79908     
[49] SS18L2|51188     ABCA7|10347      PAFAH1B3|5050   
20531 Levels: ?|100130426 ?|100133144 ?|100134869 ?|10357 ?|10431 ... ZZZ3|26009

## Heatmap

Heatmaps are visualizations that represent the level of a variable across different samples or groups using different colors to represent levels.

In this particular example, we would like to represent the different expression levels of some of the genes selected above between normal and tumor samples. 

We will represent tumor samples in red and normals as blue at the top.

First we are going to select the top 100 genes ranked, and associate the class values 0/1 in a row at the bottom of mrnaSetClass.

Because selection by column is easier, we transpose the dataframe (inverse tows and columns).

We then separate the normals and the tumors in two datasets, selecting only 112 tumors so that we have an equal number of normals and tumors.

We then combine the two sets together.

We draw the heatmap by selecting several options:
 
1) we choose a color scheme as red for tumors and blue for normals for the top bar.

2) we apply the color scheme to the classes in the 'both' dataframe.

3) we draw the heatmp with options 'scale = "row"' to normalize the rows (gene values), 'col=topo.colors(100)' as a color scheme for the heatmap cells, and 'ColSideColors=groupColors' to specify the colors chosen for the side bars.

What we find on the heatmap is that it finds several groupins in the patients, mostly two groups of normals, and two groups of tumors. So this is interesting. We can also see some gene expressions characterizing the groups.


In [1]:
# cell #11
# merge class values into mrnaNorm as the last row  - after the genes and select only top 100 genes
mrnaSetClass <- rbind(mrnaNorm[bss$ix[1:100],-1], setNames(mrnaClassNum, names(mrnaNorm[,-1])))
dim(mrnaSetClass)
# 101  1212       101 genes as rows and 1212 patients as columns
# to select by class, transpose the matrix
transpose <- as.data.frame(t(mrnaSetClass))
colnames(transpose)[101] <- "class"
# select normals
normals <- subset(transpose, transpose$class == 0)
dim(normals)
#[1] 112 101
# select tumors
tumors <- subset(transpose, transpose$class == 1)
dim(tumors)
#[1] 1100 101
# combine them together
both <- rbind(normals, tumors[1:112,])
dim(both)
# select a color scheme - red for tumor and blue for normal
color.map <- function(class) { if (class==0) "#FF0000" else "#0000FF" } # red for tumor and blue for normal
# create heatmap
groupColors <- unlist(lapply(both$class, color.map))
heatmap(as.matrix(t(both)), scale = "row", col=topo.colors(100), ColSideColors=groupColors)

ERROR: Error in rbind(mrnaNorm[bss$ix[1:100], -1], setNames(mrnaClassNum, names(mrnaNorm[, : object 'mrnaNorm' not found


ERROR: Error in eval(expr, envir, enclos): object 'mrnaSetClass' not found


ERROR: Error in t(mrnaSetClass): object 'mrnaSetClass' not found


ERROR: Error in colnames(transpose)[101] <- "class": object 'transpose' not found


ERROR: Error in subset(transpose, transpose$class == 0): object 'transpose' not found


...

ERROR: Error in eval(expr, envir, enclos): object 'tumors' not found


ERROR: Error in rbind(normals, tumors[1:112, ]): object 'normals' not found


ERROR: Error in eval(expr, envir, enclos): object 'both' not found


ERROR: Error in lapply(both$class, color.map): object 'both' not found


ERROR: Error in t(both): object 'both' not found
